In [1]:
import numpy as np
import pandas as pd
import mysql.connector
import gc, os
import seaborn as sns
import networkx as nx
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
%matplotlib inline

In [2]:

cnx = mysql.connector.connect(user='root', password='shawn672000',
                              host='localhost',
                              database='enron')
g = cnx.cursor()

q =  "SELECT * FROM message"
msgs = pd.read_sql(con=cnx, sql=q)

q = "SELECT * FROM recipientinfo"
recp = pd.read_sql(con=cnx, sql=q)


q = """SELECT DISTINCT * FROM
(SELECT concat(firstName, ' ', lastName) AS employee_name, Email_id as email_id, status AS position FROM enron.employeelist UNION
SELECT concat(firstName, ' ', lastName) AS employee_name, Email2 as email_id, status AS position FROM enron.employeelist UNION
SELECT concat(firstName, ' ', lastName) AS employee_name, Email3 as email_id, status AS position FROM enron.employeelist UNION
SELECT concat(firstName, ' ', lastName) AS employee_name, Email4 as email_id, status AS position FROM enron.employeelist) s"""
emp = pd.read_sql(con=cnx, sql=q)

msgs = msgs.merge(emp, how="inner", right_on="email_id", left_on="sender")

msgs.columns = ["mid", 'sender', 'date', 'message_id', 'subject', 'body', 'folder', 'sender_name', 'email_id', 'sender_position']

recp = recp.merge(emp, how = "inner", left_on="rvalue", right_on="email_id")

recp.columns = ["rid", "mid", "rtype", "rvalue", "dater", "recip_name", "recip_email_id", "recip_position"]

msgs = msgs.merge(recp, how="inner", left_on="mid", right_on="mid", suffixes=["_msg", "_rec"])

msgs = msgs[['sender_name', 'sender_position', 'rtype', 'recip_name', 'recip_position']].reset_index(drop=True)
del emp, recp
gc.collect()

#Who are the important people here? central people?
msgs = msgs[msgs.rtype=="TO"].reset_index(drop=True)
msgs.drop("rtype", axis=1, inplace=True)

msgs["weight"] = 0
msgs = msgs.groupby(by=["sender_name", "sender_position", "recip_name", "recip_position"]).count().reset_index()

G = nx.DiGraph()
for i in range(len(msgs)):
    sn = msgs.iloc[i]["sender_name"]
    sp = msgs.iloc[i]["sender_position"]
    rn = msgs.iloc[i]["recip_name"]
    rp = msgs.iloc[i]["recip_position"]
    w = msgs.iloc[i]["weight"]   
    G.add_node(sn, position=sp)
    G.add_node(rn, position=rp)
    G.add_edge(sn, rn, weight=w)


In [ ]:
position = pd.Series(name="position")
for n, at in G.nodes(data=True):
    position.loc[n] = at["position"]
metrics = list()
metrics.append(position)
metrics.append(pd.Series(nx.in_degree_centrality(G), name="c_in_degree"))
metrics.append(pd.Series(nx.out_degree_centrality(G), name="c_out_degree"))
metrics.append(pd.Series(nx.closeness_centrality(G), name="c_closeness"))
metrics.append(pd.Series(nx.betweenness_centrality(G, normalized=True, endpoints=False, weight="weight"), name="c_betweenness"))
metrics.append(pd.Series(nx.pagerank(G, weight="weight", alpha=0.8), name="rank_scaled"))
h, a = nx.hits(G)
metrics.append(pd.Series(h, name="hub"))
metrics.append(pd.Series(a, name="auth"))
metrics.append(pd.Series(nx.clustering(G), name="clustering"))
metrics.append(pd.Series(nx.neighbor_degree.average_neighbor_degree(G, weight=True), name="average_neigh_deg"))

metrics = pd.concat(metrics, axis=1)

In [18]:
metrics['average_neigh_deg_norm'] = MinMaxScaler().fit_transform(metrics.average_neigh_deg.values.reshape(-1, 1)).ravel()

In [4]:
edges_df = pd.DataFrame(columns=["source", "destination", "weight"])
for s, d, at in G.edges(data=True):
    edges_df.loc[len(edges_df)] = (s, d, at["weight"])
    

In [19]:
deg_cl = ['c_in_degree', 'c_out_degree']
close_and_between = ['c_closeness', 'c_betweenness']
pr = ['rank_scaled'] 
hits = ['hub', 'auth']
form_a_community = ['clustering', 'average_neigh_deg_norm']

In [21]:
km = KMeans(n_clusters=4)
km.fit(metrics[deg_cl].values)
metrics["degree_clus"] = km.labels_

km = KMeans(n_clusters=4)
km.fit(metrics[close_and_between].values)
metrics["close_and_between_clus"] = km.labels_

km = KMeans(n_clusters=4)
km.fit(metrics[hits].values)
metrics["hub_auth_clus"] = km.labels_


km = KMeans(n_clusters=4)
km.fit(metrics[form_a_community].values)
metrics["form_a_comunity"] = km.labels_




In [24]:
metrics.to_excel("enron_metrics.xlsx")
edges_df.to_excel("enron_edges.xlsx")